 ## Discussion 10A, Spring 2021
 ### Towards Upper-Triangularization By An Orthonormal Basis

Welcome! The discussion 10A worksheet presents arguments that are largely symbolic, but it can be helpful to ground the analysis in a numerical example. 

Run the code cells below for each part to follow along with the given example matrix, $S$. This square matrix is referred to as $M$ in the worksheet.

In [ ]:
import numpy as np

In [ ]:
### defining the given S matrix
S = np.array([
        [5/12, 5/12, 1/6],
        [5/12, 5/12, 1/6],
        [1/6, 1/6, 2/3]
])

print("S is: \n\n", S)

### [Part (a)] Computing the Basis $U$

In [ ]:
### defining a function to perform gram-schmidt on a given vector:
def extend_to_basis(v1: np.ndarray) -> np.ndarray:
    """
    This function uses Gram Schmidt algorithm to extend a n-by-1 column vector v1 to an
    orthonormal basis for R^n
    :param v1: n-by-1 column vector
    :return: n-by-n matrix, orthonormal basis
    """
    ncol = v1.shape[0]
    v1 = np.reshape(v1, (ncol, 1))
    # make matrix vmat with v1 and standard basis vectors
    vmat = np.hstack((v1, np.eye(ncol)))
    # initialize final orthonormal basis matrix
    qmat = np.hstack((v1 / np.linalg.norm(v1), np.zeros((ncol, ncol - 1))))
    idx = 1
    while idx < ncol:
        # Perform Gram Schmidt algorithm
        vvec = vmat[:, idx]
        zvec = vvec.copy()
        for jdx in range(idx):
            zvec -= (vvec.T @ qmat[:, jdx]) * qmat[:, jdx]
        if np.isclose(zvec, qmat[:, idx]).all():
            # this vector is not linearly independent, so delete it
            vmat = np.delete(vmat, idx, 1)
        else:
            qmat[:, idx] = zvec / np.linalg.norm(zvec)
            idx += 1
    return qmat

### calling the given function on our starting vector u, and assigning the result to the matrix U
vec_u = np.array([1, -1, 0])
U = extend_to_basis(vec_u)

print("U is: \n\n", U)

### [Part (b)] Computing $U^\top M U$ (in the worksheet, $U^\top S U$)

In [ ]:
### multiplying the matrices in the required order, and rounding to avoid floating point error.
print("U^T M U is: \n\n", np.around(U.T @ S @ U, 4))

### defining R for later subparts
R = U[:,1:]
print("\nthe R matrix is \n\n", R)

Note that the values here are the decimal equivalent of the fractions in the worksheet solutions. The fractional form is:
$$
U^\top S U =
            \begin{bmatrix}
            	0 & 0 & 0\\
             	0 & \frac{5}{6} & \frac{\sqrt{2}}{6}\\
             	0 & \frac{\sqrt{2}}{6} & \frac{2}{3}
         	\end{bmatrix}
$$

### [Part (c)] Checking that $U^\top = U^{-1}$

In [ ]:
### print transpose of U
print("transpose of U: \n\n", U.T)

In [ ]:
### print inverse of U
print("inverse of U: \n\n", np.linalg.inv(U))

Notice the results above are the same!

### [Part (d)] Checking that $S = U^\top \underbrace{\begin{bmatrix}
            \lambda_1 & \vec{a}^\top \\
            \vec{0} & Q
        \end{bmatrix}}_{T} U$, part (d)

In [ ]:
### print the left-hand side, which is just S
print("LHS matrix: S is \n\n", np.around(S, 4))

In [ ]:
### print the right-hand side, which is U^T T U, with T defined as above
v1 = np.array([1, -1, 0])/np.sqrt(2)
lambda1 = 0
a = R.T @ S.T @ v1 ### a happens to be zero here, since S was symmetric. Try defining a different S and see what happens!
Q = R.T @ S @ R
print("Q matrix is: \n\n", Q, "\n")

T = np.vstack([
    np.hstack([lambda1, a.T]),
    np.hstack([np.zeros((2, 1)), Q])
])

print("the T matrix is: \n\n", np.around(T, 4), "\n")

print("RHS matrix: \n\n", np.around(U @ T @ U.T, 4))

Note that the RHS and LHS match!

### [Part (e)] Showing that $$M =
        \begin{bmatrix} \vec{v}_1 & R\vec{v}_2 & RY \end{bmatrix}
        \begin{bmatrix} \lambda_1 & a_1 & \vec{a}_\text{rest}^\top \\
            0 & \lambda_2 & \vec{b}^\top    \\
            \vec{0} & 0 & P
        \end{bmatrix}
        \begin{bmatrix} \vec{v}_1 & R\vec{v}_2 & RY \end{bmatrix}^\top$$

In [ ]:
### First, we can list the eigenvalues and eigenvectors of Q to choose
np.linalg.eig(Q)

We take the pair $$(\lambda_2, \vec{v}_2) = \left(1, \begin{bmatrix} -0.57735027 \\ 0.81649658 \end{bmatrix}\right) = \left(1, \begin{bmatrix} \frac{-\sqrt{3}}{3} \\ \frac{\sqrt{6}}{3} \end{bmatrix}\right)$$

Then, we can make sure that Q actually matches the given form, by defining $Y$ and then carrying out the computation

In [ ]:
lambda2 = 0.5
v2 = np.array([[-np.sqrt(3)/3],  [np.sqrt(6)/3]])
Y = np.array([[np.sqrt(6)/3], [np.sqrt(3)/3]])
print("Y, which is the orthonormal extension of v2, is: \n\n", Y)

In [ ]:
print("Q is: \n\n", Q, "\n")

In [ ]:
### first we define the v2, Y concatenated matrix
v2Y = np.hstack([v2, Y])
print("the [v2, Y] matrix is: \n\n", v2Y, "\n")

### Now, we define the P matrix and b vector, analogous to what we did before
P = (Y.T @ Q @ Y).reshape(1,)
b = (Y.T @ Q.T @ v2).reshape(1,)

### Now we define the middle matrix of Q, called T2
T2 = np.vstack([
    np.hstack([lambda2, b.T]),
    np.hstack([0, P])
])

print("The P matrix is: \n\n", np.around(P, 4), "\n")

print("The middle matrix, T2, is: \n\n", np.around(T2, 4), "\n")

print("the RHS of the Q expression is: \n\n", v2Y @ T2 @ v2Y.T, "\n")

The left and right sides of the given equation match! Now let's show that our expression for "S" holds as well:

In [ ]:
print("the S matrix is: \n\n", S, "\n")

In [ ]:
### we define the block [v1, Rv2, RY]
S_blk = np.hstack([v1.reshape(3,1), R @ v2, R @ Y])

### We form the middle matrix for S
S_mid = np.vstack([
    np.hstack([lambda1, a[0], a[1:].T]),
    np.hstack([0, lambda2, b.T]),
    np.hstack([np.zeros((1,)), 0, P[0]])
])

### We print out the calculated  matrix
print("the RHS matrix is: \n\n", S_blk @ S_mid @ S_blk.T, "\n")

Wonderful! We have now expressed the original $S$ matrix in an upper-triangular form using orthonormalization